In [1]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

In [2]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [3]:
import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn.preprocessing import MaxAbsScaler

def peaks(grid):
    x = np.linspace(-2, 2, grid)
    y = np.linspace(-2, 2, grid)
    x, y = np.meshgrid(x, y)
    z = 3*(1-x)**2 * np.exp(-(x**2) - (y+1)**2) - 10*(x/5 - x**3 - y**5) * np.exp(-x**2 - y**2) - 1/3 * np.exp(-(x+1)**2 - y**2)
    return x, y, z

def create_dataframe(grid):
   x, y, z = peaks(grid)
   data = {'X': x.flatten(), 'Y': y.flatten(), 'Z': z.flatten()}
   df = pd.DataFrame(data)
   return df

In [4]:
df_25 = create_dataframe(grid=5)
df_1000 = create_dataframe(grid=32)

In [5]:
def split_df(df):
    # df = df.sample(frac=1).reset_index(drop=True)  # Randomiza as linhas
    _input = np.vstack([df['X'], df['Y']]).T
    _output = np.array(df['Z'])
    return (_input, _output)

In [6]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import levenberg_marquardt as lm
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error
from keras import regularizers
from keras import initializers

class TrainWithSmallDataset:
    def __init__(self, batch_size=1000, store=25):
        self.batch_size = batch_size
        self.betters = []
        self.store = store

    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)


    def split_dataset(self, input, output, sup_input, sup_output):

      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.sup_dataset, self.sup_input, self.sup_output = self.create_dataset(sup_input, sup_output)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)

    def create_model(self, input_size=2, sumary=False):
      regularizer = regularizers.L2(0.2)
      initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

      self.model = tf.keras.Sequential([
          tf.keras.layers.Dense(30,
                                activation='tanh',
                                input_shape=(input_size,),
                                kernel_regularizer=regularizer,
                                kernel_initializer=initializer,
                                ),
          tf.keras.layers.Dense(16,
                                activation='tanh',
                                kernel_regularizer=regularizer,
                                kernel_initializer=initializer,
                                ),
          tf.keras.layers.Dense(1,
                                activation='linear',
                                kernel_regularizer=regularizer,
                                kernel_initializer=initializer,
        )])

      if (sumary == True):
        self.model.summary()

      self.model.compile(
          optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
          loss=tf.keras.losses.MeanSquaredError())

      self.lm_model = lm.ModelWrapper(
          tf.keras.models.clone_model(self.model))

      self.lm_model.compile(
          optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
          loss=lm.MeanSquaredError())

    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.sup_prediction = self.lm_model.predict(self.sup_input)
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()
          sup_pred = self.sup_prediction.flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)
          r2_sup = r2_score(self.sup_output, sup_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)
          mse_sup = mean_squared_error(self.sup_output, sup_pred)

          metrics = {
                          'r2': r2,
                          'r2_sup': r2_sup,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_sup': mse_sup,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics


    def plot_results(self):
      fig = plt.figure(figsize=(18, 6))
      ax0 = fig.add_subplot(1, 3, 1, projection='3d')
      ax2 = fig.add_subplot(1, 3, 2, projection='3d')
      ax1 = fig.add_subplot(1, 3, 3)

      ax0.set_title('Superficie Fit')
      ax0.scatter(self.sup_input[:, 0], self.sup_input[:, 1], self.sup_prediction, c='r', marker='s', label="lm")
      ax0.scatter(self.sup_input[:, 0], self.sup_input[:, 1], self.sup_output, c='b', marker='o', label="reference")
      ax0.legend()

      ax2.set_title('Teste Fit')
      ax2.scatter(self.test_input[:, 0], self.test_input[:, 1], self.test_prediction, c='r', marker='s', label="lm")
      ax2.scatter(self.test_input[:, 0], self.test_input[:, 1], self.test_output, c='b', marker='o', label="reference")
      ax2.legend()

      ax1.plot(self.results.history['loss'], label='Erro Treino')
      ax1.plot(self.results.history['val_loss'], label='Erro validação')
      ax1.set_title('Histórico de Treinamento')
      ax1.set_ylabel('Função de Custo')
      ax1.set_xlabel('Épocas de Treinamento')
      ax1.legend(['Erro Treino', 'Erro validação'])

      return(fig)

2024-01-29 23:27:16.583197: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-29 23:27:18.806564: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-29 23:27:18.806656: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-29 23:27:19.129081: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-29 23:27:19.858200: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-29 23:27:19.859114: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [7]:
from keras import saving
from keras import models
import pickle

class Tester:
  def __init__(self, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input_25, self.output_25 = split_df(df_25)
    self.input_1000, self.output_1000 = split_df(df_1000)

  def PrepareTraining(self):
    trainer = TrainWithSmallDataset()
    trainer.split_dataset(self.input_25, self.output_25, self.input_1000, self.output_1000)
    return (trainer)

  def Train(self, trainer, epochs=1000):
    trainer.create_model()
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path)
  
  def SaveFigs(self, trainer, fileName):
    path = f"../results/{fileName}.png" 
    open(path,'w').close()
    figure = trainer.plot_results()
    figure.savefig(path)
    plt.close(figure)

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)
  
  def LoadDataset(self, fileName, trainer=None):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'rb') as f:
      _train, _vt, _val, _test = pickle.load(f)
    # trainer._train, trainer._vt, trainer._val, trainer._test = train, vt, val, test
    print(_train, _vt, _val, _test)

  # brief: for each dataset
  def Loop(self, sort_by, boundarie):
    better_model = 0

    for j in range(self.dataset_run_times):
      trainer = self.PrepareTraining() # split dataset
      self.SaveDataset(trainer, f"dataset_{j}")

      for i in range(self.run_times):
        print (f"+++++++++++ Dataset [{j + 1}] | {i + 1} ++++++++++++++++++")
        metrics, model = self.Train(trainer)
        if (metrics[sort_by] <= boundarie): # should be >= to acsending metrics
          fileName = f"model_{j}_{better_model}"
          self.SaveModelWeights(model, fileName)
          self.SaveFigs(trainer, fileName)
          self.better_metrics[fileName] = metrics
          better_model += 1
              

  def DisplayBetterResults(self, sort_by):
    df = pd.DataFrame.from_dict(self.better_metrics)
    df = df.transpose()
    df = df.sort_values([sort_by])
    display(df)
    excel_path = '../results/metrics.xlsx'
    df.to_excel(excel_path, index=False)
    print(f"DataFrame salvo em {excel_path}")

# Treinando com 25 dados


In [8]:
tester = Tester(run_times=10, dataset_run_times=100)

In [9]:
tester.Loop(sort_by='mse_sup', boundarie=2.5)

+++++++++++ Dataset [1] | 1 ++++++++++++++++++
Elapsed time:  0.9175864879998699
Stopped at epoch:  7
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [1] | 2 ++++++++++++++++++
Elapsed time:  0.46021592599981886
Stopped at epoch:  8
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [1] | 3 ++++++++++++++++++
Elapsed time:  0.4421759739998379
Stopped at epoch:  8
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [1] | 4 ++++++++++++++++++
Elapsed time:  0.40048834000026545
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [1] | 5 ++++++++++++++++++
Elapsed time:  0.5715799399999923
Stopped at epoch:  14
1/1 [==============================] - 0s 16ms/step
+++++++++++ Dataset [1] | 6 ++++++++++++++++++
Elapsed time:  0.4263929950002421
Stopped at epoch:  8
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [1] | 7 ++++++++++++++++++
Elapsed time:  0.450876012

2024-01-29 23:29:09.417232: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


Elapsed time:  24.42677709599957
Stopped at epoch:  0
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [3] | 5 ++++++++++++++++++
Elapsed time:  24.593832874999862
Stopped at epoch:  0
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [3] | 6 ++++++++++++++++++
Elapsed time:  24.317603278000206
Stopped at epoch:  0
1/1 [==============================] - 0s 16ms/step
+++++++++++ Dataset [3] | 7 ++++++++++++++++++
Elapsed time:  23.999115803000223
Stopped at epoch:  0
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [3] | 8 ++++++++++++++++++
Elapsed time:  24.244070614000066
Stopped at epoch:  993
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [3] | 9 ++++++++++++++++++
Elapsed time:  7.549579327999709
Stopped at epoch:  301
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [3] | 10 ++++++++++++++++++
Elapsed time:  25.00417517599999
Stopped at epoch:  0
1/1 [==============

In [10]:
tester.DisplayBetterResults('mse_sup')

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_12_3,0.912817,0.709965,0.789034,0.787231,0.809304,0.365887,1.939464,0.830575,1.456219,1.143397
model_86_20,0.896575,0.689825,0.958560,0.611721,0.707947,0.434054,2.074143,0.043276,1.481779,0.762527
model_32_12,0.843180,0.686478,0.713144,0.766650,0.768147,0.658139,2.096526,0.635646,3.477723,2.056684
model_94_21,0.861332,0.671307,0.703001,0.590693,0.607494,0.581959,2.197973,0.217619,2.837029,1.527324
model_38_16,0.856884,0.671065,0.775598,0.470174,0.619945,0.600628,2.199590,0.544178,3.145496,1.844837
model_32_11,0.817672,0.669968,0.733347,0.718752,0.730434,0.765193,2.206926,0.590878,4.191576,2.391227
model_12_4,0.882245,0.656542,0.500607,0.835977,0.742433,0.494193,2.296708,1.966113,1.122593,1.544353
model_7_0,0.879245,0.654805,0.733049,0.669689,0.718666,0.506783,2.308319,0.271344,1.864515,1.067930
model_7_1,0.863382,0.644982,0.868800,0.665572,0.733781,0.573355,2.374004,0.133359,1.887753,1.010556
model_16_6,0.731483,0.643319,0.395428,0.691814,0.590159,1.126908,2.385126,4.740208,2.302968,3.521588


DataFrame salvo em ../results/metrics.xlsx
